### Table 4 Parameter sensitivity

In [1]:
from __future__ import print_function
import numpy as np
import os
import sys
sys.path.append('./scripts/')
import pandas as pd
import xarray as xr
import cmaps
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from isca_cre_cwp import get_gm, calc_toa_cre_for_isca, calc_total_cwp_for_isca, add_toa_net_flux_to_ds_arr

In [2]:
def get_model_flux_arr(ds):
    # ---------- TOA -------------- #
    toa_net_sw = get_gm(ds.soc_toa_sw)
    olr = get_gm(ds.soc_olr)
    toa_net_flux = toa_net_sw - olr

    toa_sw_cre = get_gm(ds.toa_sw_cre)
    toa_lw_cre = get_gm(ds.toa_lw_cre)
    toa_net_cre = get_gm(ds.toa_net_cre)

    cwp = get_gm(ds.cwp) * 1e3

    mod_arr = [toa_net_sw, olr, toa_net_flux,
                toa_sw_cre, toa_lw_cre, toa_net_cre, cwp,]
    return mod_arr

def get_model_cf_arr(ds):
    try:
        low_ca = get_gm(ds.low_cld_amt_mxr)
        mid_ca = get_gm(ds.mid_cld_amt_mxr)
        high_ca = get_gm(ds.high_cld_amt_mxr)
        tot_ca = get_gm(ds.tot_cld_amt_mxr)
    except:
        low_ca = get_gm(ds.low_cld_amt)
        mid_ca = get_gm(ds.mid_cld_amt)
        high_ca = get_gm(ds.high_cld_amt)
        tot_ca = get_gm(ds.tot_cld_amt)
    mod_arr = [low_ca, mid_ca, high_ca, tot_ca]
    return mod_arr

In [3]:
def print_property_table_in_exps(ds_arr, ds_names, file_name=None, float_fmt='%.2f', add_cf=True):
    
    row_names = [r'Low cloud amount', r'Middle cloud amount',
                 r'High cloud amount', r'Total cloud amount',
                 r'TOA net SW flux', r'TOA net LW flux',
                 r'TOA net flux',
                 r'TOA SW CRE', r'TOA LW CRE',
                 r'TOA net CRE', r'Cloud water path']
    if not add_cf:
        row_names = row_names[4:]
        
    N = len(ds_arr)
    col_names = []
    for ds_nm in ds_names:
        col_names.append(ds_nm)

    table = np.zeros((len(row_names), N), dtype='double')

    for nn, ds in enumerate(ds_arr):
        mod_flux_arr = get_model_flux_arr(ds)
        if add_cf:
            mod_cf_arr = get_model_cf_arr(ds)
            mod_arr = mod_cf_arr + mod_flux_arr
        else:
            mod_arr = mod_flux_arr
        for i, dt in enumerate(mod_arr):
            table[i, nn] = dt

    tbl = pd.DataFrame(data=table, index=row_names, columns=col_names)
    if file_name is None:
        print(tbl.to_latex(float_format=float_fmt))
    else:
        tbl.to_latex(buf=file_name, float_format=float_fmt)
    return tbl

In [4]:
P = os.path.join
saved_dt_dir = './data'
isca_output_dir = './data/isca_output'

exp_files = []
exp_names = []
ds_arr = []

# ============================= default ========================== #
exp_files.append('socrates_realistic_continent_amip_fixed_SST_with_clouds_linear_ALL.nc')
exp_names.append('default')

# ============================= vary effective radius ========================== #
exp_base_nm = 'sensitivity_test_amip_fixed_sst_linear_clouds'

reff_liq_arr = [16, 12]
# reff_ice = 25
for reff_liq in reff_liq_arr:
    exp_fn = exp_base_nm + '_reff_liq_' + str(reff_liq) + '.nc'
    exp_files.append(exp_fn)
    exp_names.append('re_liq=' + str(reff_liq))

reff_ice_arr = [30]
# reff_liq = 14
for reff_ice in reff_ice_arr:
    exp_fn = exp_base_nm + '_reff_ice_' + str(reff_ice) + '.nc'
    exp_files.append(exp_fn)
    exp_names.append('re_ice=' + str(reff_ice))

# ============================= vary qcl ========================== #
qcl_arr = [0.15,]
for qcl_val in qcl_arr:
    exp_fn = exp_base_nm + '_qcl_' + str(qcl_val) + '.nc'
    exp_files.append(exp_fn)
    exp_names.append('qcl='+str(qcl_val))

for exp_fn in exp_files:
    print(exp_fn)
    ds = xr.open_dataset(P(isca_output_dir, exp_fn), decode_times=False)
    ds_arr.append(ds)

print('Calculating related flux and CRE...')
# calculate TOA CRE for Isca
calc_toa_cre_for_isca(ds_arr)

# calculate cloud water path for Isca
calc_total_cwp_for_isca(ds_arr)

add_toa_net_flux_to_ds_arr(ds_arr)
print('Calculation finished.')

socrates_realistic_continent_amip_fixed_SST_with_clouds_linear_ALL.nc
sensitivity_test_amip_fixed_sst_linear_clouds_reff_liq_16.nc
sensitivity_test_amip_fixed_sst_linear_clouds_reff_liq_12.nc
sensitivity_test_amip_fixed_sst_linear_clouds_reff_ice_30.nc
sensitivity_test_amip_fixed_sst_linear_clouds_qcl_0.15.nc
Calculating related flux and CRE...
Calculation finished.


In [5]:
print('Saving the table...')
file_name = P(saved_dt_dir, 'sensivity_table_linear.tex')
tbl = print_property_table_in_exps(ds_arr, exp_names, file_name=file_name, float_fmt='%.1f', add_cf=False)
print(file_name + ' saved.')

Saving the table...
./data/sensivity_table_linear.tex saved.


In [6]:
tbl

,default,re_liq=16,re_liq=12,re_ice=30,qcl=0.15
TOA net SW flux,230.625959,240.252442,234.661118,238.919186,241.411235
TOA net LW flux,227.071707,228.957756,228.641775,229.845611,229.688731
TOA net flux,3.554252,11.294686,6.019343,9.073575,11.722505
TOA SW CRE,-57.345520,-47.755637,-53.326291,-49.069297,-46.586617
TOA LW CRE,31.630272,30.245916,30.337533,29.335178,29.463338
TOA net CRE,-25.715247,-17.509720,-22.988759,-19.734119,-17.123278
Cloud water path,127.822991,116.268089,116.170079,115.415830,96.433432
